In [76]:
import pandas as pd
import requests
import os
import json
from datetime import datetime
import uuid

In [2]:
consumer_key= 'p5CzEB9O7j6uS4pgoWGpG28oe'
consumer_secret= 'Q8lMvPsnuKFJ7WmCRBbX2ZuIyD6kCg21oHjnE41kAXWv9V0FmN'
access_token= '1097726291065954304-ltQ5XhlGij1Tq90YtQdli9u1N4y0j6'
access_token_secret= 'GYmiQKmmcLk1xwc1M5yQxs9i9I7uMBNMVqfaVJF9GvDFh'

In [101]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAPrxMQEAAAAArdIifiX3DO3CXHsKN33dVku4TGw%3DftFN1DlKV5HNLKmKB7KFwEfiPONZqx9oRt8AfAtGkyjV48bsN1'

### Scenario 1: Real-time streaming

***Note:*** The run has been terminated after extracting two tweets since this is real-time continuous run

In [100]:
import requests
import os
import json

df=pd.DataFrame()

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r

def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "Justin Beiber", "tag": "music"}
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            
            #Loading to Bigquey
            
            print(json.dumps(json_response, indent=4, sort_keys=True))
            df['id']=[json_response['data']['id']]
            df['text']=[json_response['data']['text']]
            #Extracting current timestamp
            currentdt=datetime.now()
            df['load_date_time']=[currentdt]

            #Creating unique batch id
            id = uuid.uuid1()
            df['BatchId']=[id.int]
            df.to_gbq('myproject-330700.twitter_data.tweet_data', project_id='myproject-330700',credentials=credentials,
                                  if_exists='append')


def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)


if __name__ == "__main__":
    main()

{"data": [{"id": "1454882244762509312", "value": "Justin Beiber", "tag": "music"}], "meta": {"sent": "2021-10-31T19:01:34.815Z", "result_count": 1}}
{"meta": {"sent": "2021-10-31T19:01:36.053Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "Justin Beiber", "tag": "music", "id": "1454886289078108168"}], "meta": {"sent": "2021-10-31T19:01:37.757Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
200
{
    "data": {
        "id": "1454886797197021191",
        "text": "AAAAH JUSTIN BEIBER"
    },
    "matching_rules": [
        {
            "id": "1454886289078108168",
            "tag": "music"
        }
    ]
}


1it [00:03,  3.26s/it]


{
    "data": {
        "id": "1454887602167992328",
        "text": "@OldMemeArchive F**K YOU JUSTIN BEIBER"
    },
    "matching_rules": [
        {
            "id": "1454886289078108168",
            "tag": "music"
        }
    ]
}


1it [00:03,  3.61s/it]


KeyboardInterrupt: 

### Loading Stream into Bigquery

![tweet data](tweet_stream_load.png)


***Since this is real-time load into bigquery we cannot capture unique count/total count. Those were captured as part of Batch Load Scenario Below***

### Scenario 2: Batch Load

***Note:*** The free account which has been utilized for this exercise will extract all tweets related to keyword for past 7 days from a given date

In [20]:
import tweepy 

In [21]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [62]:
word='justin beiber #music'

In [66]:
tweets = tweepy.Cursor(api.search_tweets,
              q=word,
              lang="en",
              until="2021-10-30").items()
tweets

In [67]:
data=[[tweet.id,tweet.created_at,tweet.text] for tweet in tweets]

In [68]:
data

[[1453371085563572230,
  datetime.datetime(2021, 10, 27, 14, 40, 43, tzinfo=datetime.timezone.utc),
  '#NowPlaying: Runaway Love (Remix) (Feat. Raekwon &amp; Kanye West) by Justin Beiber | Tune in to #SexyBlackRadio (link… https://t.co/5WQ3dd9506'],
 [1452358382699044871,
  datetime.datetime(2021, 10, 24, 19, 36, 36, tzinfo=datetime.timezone.utc),
  'Arianna Grande\nThe Weeknd\nDrake\nJeremih\nPolo G\nThe Kid Laroi\nPost Malone\nJustin Beiber\nDJ Khaled\n\n🙏🙏🙏🙏🙏🙏🙏 #music']]

In [91]:
df=pd.DataFrame()
ids=[data[i][0] for i in range(0,len(data))]
text=[data[i][1] for i in range(0,len(data))]

df['id']=ids
df['text']=text

#count of all tweets extracted
twcount=len(df)
#count of unique tweets extracted
twuncount=len(df.id.unique())

#Dropping duplicates
df.drop_duplicates(subset ="id",keep = False, inplace = True)

#Extracting current timestamp
currentdt=datetime.now()
df['load_date_time']=currentdt

#Creating unique batch id
id = uuid.uuid1()
df['BatchId']=id.int

#Creating log table
log_df=pd.DataFrame()
log_df['BatchId']=[id.int]
log_df['tweet_count']=[twcount]
log_df['uniq_tweet_count']=[twuncount]
log_df['load_date_time']=[currentdt]

### Loading to Google Bigquery

In [92]:
from google.oauth2 import service_account
service_account_info = json.load(open('/Users/prudhvichandra/Documents/twitter_gcp_scacc.json'))
credentials = service_account.Credentials.from_service_account_info(
    service_account_info)

In [93]:
df.to_gbq('myproject-330700.twitter_data.tweet_data', project_id='myproject-330700',credentials=credentials,
          if_exists='append')

1it [00:06,  6.15s/it]


In [94]:
log_df.to_gbq('myproject-330700.twitter_data.tweeet_log', project_id='myproject-330700',credentials=credentials,
          if_exists='append')

1it [00:05,  5.37s/it]


### Link to access tweet data on bigquery 

![tweet data](tweet_data.png)

### Link to access log table of tweet Load

![tweet data](tweet_log.png)


### Questions:

***What are the risks involved in building such a pipeline?***

- Need to have effecient failure handling mechanisms otherwise there is high chance of missing data or having duplicated data
- The API token, keys and urls has to closely monitored and have to updated when there is a change
- Error traceability is an important aspect of such pipelines

***How would you roll out the pipeline going from proof-of-concept to a production-ready solution?***

Inorder to roll out a production ready solution we need to determine the following:

- Agree upon how frequently we need this data to be stored(real-time/Batch) based on business requirement.
- Once agreed upon we have to collaborate with API teams and make sure we have all the production ready urls and keys required
- Then we need to design a Data Architecture and define what components or services are needed to accomplish this
    - API ----> App running on Google Cloud Run/VM to stream data from api -----> Load into Bigquery(Real-time)
    - API ----> Python code on Google Cloud Functions that can be scheduled using pubsub & Cloud Scheduler -------> Loading into Bigquery(Batch)
- Once we have data architecture finalised we define table structures/schemas and access controls etc.
- The we need to parameterise the Python code for example storing the keys, tokens, table names, urls in config files/env variables and read from there instead of hard coding. Shell scripting etc.
- Then we make sure we enable each service required and build a end-to-end pipeline on dev
- Testing the pipeline
- UAT approval
- Moving elements to production


***What would a production-ready solution entail that a POC wouldn't?***

- No defined Architecture
- Everything is hard-coded in POC, no config files, no shell sripts, no env variables
- No enabling systems to run the code
- No users are involved in POC stage
    
    
***What is the level of effort required to deliver each phase of the solution?***

- Intial Business Analysis with users/stakeholders to define product owners and agree upon the requirements
- Data Modelling/Architecture to design an architure and other enabling elements 
- Cloud Infrasture setup and code finalizations.
- Engineering data pipeline and making sure we have the business use case justified
- Testing the pipeline
- User acceptance and approval
- Deploying the pipeline into production

    
***What is your estimated timeline for delivery for a production-ready solution?***

- Businees Analysis and complete requirement handling: 1 week
- Data Modelling/ Architecture: 1 week
- Cloud Infrasture setup and code enablement: 2 weeks
- Testing: 1 week
- UAT & Deployment: 1 Week

